<a href="https://colab.research.google.com/github/StatPhysBio/protein_holography-web/blob/main/HCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Zero-shot variant effect prediction with HCNN

This notebook can be used to run HCNN - a model based on [the paper by Pun et al.](https://www.pnas.org/doi/10.1073/pnas.2300838121) - on desired protein structures.

Please see our new [github repo](https://github.com/StatPhysBio/protein_holography-web/tree/main?tab=readme-ov-file) for more information.


In [1]:
#@title <b>Install condacolab

try:
    import google.colab
    !pip install condacolab
    import condacolab
    condacolab.install()
except ModuleNotFoundError:
    pass

✨🍰✨ Everything looks OK!


In [28]:
#@title <b>Download repository, models and setup environment</b>

#@markdown <b>NOTE:</b> If prompted to restart the runtime due to some packages being already installed, simply restart the session and re-run the cells above and including this one. The conflict should resolve itself after 1 or 2 restarting prompts, without needing to re-install everything so it should be fast. We are currently working on making that not happen.

! rm -r sample_data

! pip install torch==1.13.1
! conda install zernikegrams -c william-galvin -c conda-forge

! git clone https://github.com/StatPhysBio/protein_holography-web.git
%cd protein_holography-web
! pip install .
%cd ..

import os
from glob import glob
from tqdm import tqdm

# make diretory of runs
RUNS_DIR = '/content/runs/'
os.makedirs(RUNS_DIR, exist_ok=True)

def get_next_run_number():
    rundirs = glob(os.path.join(RUNS_DIR, 'run_*'))
    if len(rundirs) == 0:
        return 1
    max_run_num = max([int(rundir.split('_')[-1]) for rundir in rundirs])
    return max_run_num + 1

# make example
with open('example_pdb_and_chain_ids.txt', 'w+') as f:
    f.write('P68871\n')
    f.write('1AO7 A\n')
    f.write('1AO7 C\n')


rm: cannot remove 'sample_data': No such file or directory
Channels:
 - william-galvin
 - conda-forge
Platform: linux-64
Solving environment: | / - \ | / done


==> WARNING: A newer version of conda exists. <==
    current version: 23.11.0
    latest version: 24.5.0

Please update conda by running

    $ conda update -n base -c conda-forge conda



# All requested packages already installed.

fatal: destination path 'protein_holography-web' already exists and is not an empty directory.
/content/protein_holography-web
Processing /content/protein_holography-web
  Preparing metadata (setup.py) ... done
  Created wheel for protein-holography-web: filename=protein_holography_web-0.1.0-py3-none-any.whl size=4458 sha256=f9594f5121d4ff0df0df41eefdcf1190c273518f15830ef98e02392096e89ec8
  Stored in directory: /root/.cache/pip/wheels/12/1f/0a/8d64a928044d97e5c62f680be31cbffbff1d88e8e480a60cab
Successfully built protein-holography-web
  Attempting uninstall: protein-holography-web
    

In [29]:
#@title <b>Specify and / or download desired structures</b>

#@markdown ### Option 1: Single structure

#@markdown Choose between <b>ONE</font></b> of the possible input sources for the target pdb.
#@markdown - AlphaFold2 PDB (v4) via Uniprot ID:
AF_ID =''#@param {type:"string"}
#@markdown - PDB ID (imported from RCSB PDB):
PDB_ID =''#@param {type:"string"}
#@markdown - Specify path to pdb
PDB_PATH ='' #@param {type:"string"}

#@markdown Select target chain. Default is "all" which computes all chains. Note that we do not remove the other chains from the PDB file.
CHAIN_ID='all' #@param {type:"string"}


#@markdown

#@markdown ### Option 2: Multiple structures
#@markdown - Path to a .txt file containing, for each row, the name of a structure, followed optionally by a space and a CHAIN_ID. See "example_pdb_and_chain_ids.txt" for an example.
PDB_AND_CHAIN_IDS ='example_pdb_and_chain_ids.txt' #@param {type:"string"}
#@markdown - (Optional) Directory where to find structures specified in PDB_AND_CHAIN_IDS. If a structure name is not found, we will attempt to download from RCSB if the name has length 4, and from the AlphaFold2 v4 database if the name has length 6 - like Uniprot IDs.
PDB_DIR ='' #@param {type:"string"}




NEXT_RUN_DIR = os.path.join(RUNS_DIR, f'run_{get_next_run_number()}')
TEMP_PDB_DIR = os.path.join(NEXT_RUN_DIR, 'pdb_dir')
TEMP_PDB_AND_CHAIN_FILE = os.path.join(NEXT_RUN_DIR, 'pdbs_and_chains.txt')
os.makedirs(NEXT_RUN_DIR)
os.makedirs(TEMP_PDB_DIR)


# option 1
if PDB_PATH != '' or AF_ID != '' or PDB_ID != '':

    if PDB_AND_CHAIN_IDS != '':
        os.system(f'rm -r {NEXT_RUN_DIR}')
        raise ValueError('Please specify only one between AF_ID, PDB_ID, PDB_PATH and PDB_AND_CHAIN_IDS')

    if PDB_PATH != '':
        if AF_ID != '' or PDB_ID != '':
            os.system(f'rm -r {NEXT_RUN_DIR}')
            raise ValueError('Please specify only one between AF_ID, PDB_ID, PDB_PATH and PDB_AND_CHAIN_IDS')
        os.system(f"mv {PDB_PATH} {TEMP_PDB_DIR}")
        filename = '.'.join(os.basename(PDB_PATH).split('.')[:-1])

    elif AF_ID !='':
        if len(AF_ID) < 6:
            os.system(f'rm -r {NEXT_RUN_DIR}')
            raise ValueError("AF_ID must have length at least 6 since it's a uniprot ID")
        if PDB_PATH != '' or PDB_ID != '':
            os.system(f'rm -r {NEXT_RUN_DIR}')
            raise ValueError('Please pecify only one between AF_ID, PDB_ID, PDB_PATH and PDB_AND_CHAIN_IDSR')
        os.system(f'curl -s -f https://alphafold.ebi.ac.uk/files/AF-{AF_ID}-F1-model_v4.pdb -o {os.path.join(TEMP_PDB_DIR, f"AF-{AF_ID}-F1-model_v4.pdb")}')
        filename = f'AF-{AF_ID}-F1-model_v4'

    elif PDB_ID !='':
        if len(PDB_ID) != 4:
            os.system(f'rm -r {NEXT_RUN_DIR}')
            raise ValueError("PDB_ID must have length 4 to be valid.")
        if PDB_PATH != '' or AF_ID != '':
            os.system(f'rm -r {NEXT_RUN_DIR}')
            raise ValueError('Please pecify only one between AF_ID, PDB_ID, PDB_PATH and PDB_AND_CHAIN_IDS')
        os.system(f'curl -s -f https://files.rcsb.org/download/{PDB_ID}.pdb -o {os.path.join(TEMP_PDB_DIR, f"{PDB_ID}.pdb")}')
        filename = PDB_ID

    if len(CHAIN_ID) > 1 and CHAIN_ID != 'all':
        os.system(f'rm -r {NEXT_RUN_DIR}')
        raise ValueError(f'CHAIN_ID must be a single characher, cannot be "{CHAIN_ID}"')

    with open(TEMP_PDB_AND_CHAIN_FILE, 'w+') as f:
        f.write(filename)
        if CHAIN_ID != 'all':
            f.write(' ' + CHAIN_ID)

# option 2
elif PDB_AND_CHAIN_IDS != '':

    with open(PDB_AND_CHAIN_IDS, 'r') as f:
        lines = f.readlines()
        names = [line.strip().split()[0] for line in lines]

    print(f'Using structures in {PDB_AND_CHAIN_IDS}...')
    with open(TEMP_PDB_AND_CHAIN_FILE, 'w+') as f:
        for name, line in tqdm(zip(names, lines), total=len(names)):
            if os.path.exists(os.path.join(PDB_DIR, name + '.pdb')): # file already exists
                os.system(f"mv {os.path.join(PDB_DIR, name + '.pdb')} {TEMP_PDB_DIR}")
                f.write(line)
            elif len(name) == 4: # download from RCSB
                os.system(f'curl -s -f https://files.rcsb.org/download/{name}.pdb -o {os.path.join(TEMP_PDB_DIR, f"{name}.pdb")}')
                if not os.path.exists(os.path.join(TEMP_PDB_DIR, f"{name}.pdb")):
                    os.system(f'rm -r {NEXT_RUN_DIR}')
                    raise ValueError(f'Could not find structure called "{name}"')
                f.write(line)
            elif len(name) == 6: # download from AlphaFold2 v4 database
                os.system(f'curl -s -f https://alphafold.ebi.ac.uk/files/AF-{name}-F1-model_v4.pdb -o {os.path.join(TEMP_PDB_DIR, f"AF-{name}-F1-model_v4.pdb")}')
                if not os.path.exists(os.path.join(TEMP_PDB_DIR, f"AF-{name}-F1-model_v4.pdb")):
                    os.system(f'rm -r {NEXT_RUN_DIR}')
                    raise ValueError(f'Could not find "{name}"')
                f.write(line.replace(name, f'AF-{name}-F1-model_v4'))
            else:
                os.system(f'rm -r {NEXT_RUN_DIR}')
                raise ValueError(f'Could not find structure called "{name}"')
    print()
    print()

else:
      os.system(f'rm -r {NEXT_RUN_DIR}')
      raise ValueError('Please specify one between AF_ID, PDB_ID, PDB_PATH and PDB_AND_CHAIN_ID')


print(f'PDB file(s) saved to {TEMP_PDB_DIR}')
print(f'Requested structure(s) and CHAIN_ID(s) saved to {TEMP_PDB_AND_CHAIN_FILE}')


Using structures in example_pdb_and_chain_ids.txt...


100%|██████████| 3/3 [00:03<00:00,  1.33s/it]



PDB file(s) saved to /content/runs/run_13/pdb_dir
Requested structure(s) and CHAIN_ID(s) saved to /content/runs/run_13/pdbs_and_chains.txt


In [25]:
#@title <b>Run selected model with desired outputs</font></b>

#@markdown <b>Choose one and only one model version<b>
HCNN_biopython_proteinnet_0p00=False#@param {type:"boolean"}
HCNN_biopython_proteinnet_0p50=False#@param {type:"boolean"}
HCNN_biopython_proteinnet_extra_mols_0p00=True#@param {type:"boolean"}
HCNN_biopython_proteinnet_extra_mols_0p50=False#@param {type:"boolean"}

if HCNN_biopython_proteinnet_0p00:
    if HCNN_biopython_proteinnet_0p50 or HCNN_biopython_proteinnet_extra_mols_0p00 or HCNN_biopython_proteinnet_extra_mols_0p50: raise ValueError("Please select one and only one model.")
    MODEL_VERSION = "HCNN_biopython_proteinnet_0p00"
elif HCNN_biopython_proteinnet_0p50:
    if HCNN_biopython_proteinnet_0p00 or HCNN_biopython_proteinnet_extra_mols_0p00 or HCNN_biopython_proteinnet_extra_mols_0p50: raise ValueError("Please select one and only one model.")
    MODEL_VERSION = "HCNN_biopython_proteinnet_0p50"
elif HCNN_biopython_proteinnet_extra_mols_0p00:
    if HCNN_biopython_proteinnet_0p00 or HCNN_biopython_proteinnet_0p50 or HCNN_biopython_proteinnet_extra_mols_0p50: raise ValueError("Please select one and only one model.")
    MODEL_VERSION = "HCNN_biopython_proteinnet_extra_mols_0p00"
elif HCNN_biopython_proteinnet_extra_mols_0p50:
    if HCNN_biopython_proteinnet_0p00 or HCNN_biopython_proteinnet_0p50 or HCNN_biopython_proteinnet_extra_mols_0p00: raise ValueError("Please select one and only one model.")
    MODEL_VERSION = "HCNN_biopython_proteinnet_extra_mols_0p50"
else:
    raise ValueError("Please select one and only one model.")

#@markdown The first two models exclude extra ligands and ions in the PDB files.
#@markdown The other two keep them. All models remove waters. \
#@markdown "_0p00" and "_0p50" denote the amount of noise - in Angstrom - with which the models were trained.

#@markdown

#@markdown <b>Choose desired outputs<b>
probas=True#@param {type:"boolean"}
logprobas=True#@param {type:"boolean"}
logits=True#@param {type:"boolean"}
embeddings=True#@param {type:"boolean"}

REQUEST_STRING_NO_EMBEDDINGS = '' # only here for printing purposes
REQUEST_STRING = ''
if probas:
    REQUEST_STRING_NO_EMBEDDINGS += ' probas'
    REQUEST_STRING += ' probas'
if logprobas:
    REQUEST_STRING_NO_EMBEDDINGS += ' logprobas'
    REQUEST_STRING += ' logprobas'
if logits:
    REQUEST_STRING_NO_EMBEDDINGS += ' logits'
    REQUEST_STRING += ' logits'
if embeddings:
    REQUEST_STRING += ' embeddings'
REQUEST_STRING_NO_EMBEDDINGS = REQUEST_STRING.strip()
REQUEST_STRING = REQUEST_STRING.strip()


import torch
if torch.cuda.is_available():
    print('Running on cuda.')
else:
    print('Running on cpu.')

# os.system(f"python protein_holography-web/run_hcnn_on_pdbfiles.py \
#             -m {MODEL_VERSION} \
#             -pd {TEMP_PDB_DIR} \
#             -pn {TEMP_PDB_AND_CHAIN_FILE} \
#             -o {os.path.join(NEXT_RUN_DIR, f'output_{MODEL_VERSION}.csv')} \
#             -v 0 \
#             -lb 1 \
#             -r {REQUEST_STRING}")

OUTPUT_FILE = os.path.join(NEXT_RUN_DIR, f'output_{MODEL_VERSION}.csv')

! python protein_holography-web/run_hcnn_on_pdbfiles.py \
            -m {MODEL_VERSION} \
            -pd {TEMP_PDB_DIR} \
            -pn {TEMP_PDB_AND_CHAIN_FILE} \
            -o {OUTPUT_FILE} \
            -v 0 \
            -lb 1 \
            -r {REQUEST_STRING}

def make_pretty_string(request_string):
    split_string = request_string.split()
    if len(split_string) == 1:
        return request_string
    else:
        return ', '.join(split_string[:-1]) + ' and ' + split_string[-1]

print(f'Saved {make_pretty_string(REQUEST_STRING_NO_EMBEDDINGS)} to {os.path.join(NEXT_RUN_DIR, f"output_{MODEL_VERSION}.csv")}.')
if embeddings:
    print(f'Saved embeddings to {os.path.join(NEXT_RUN_DIR, f"output_{MODEL_VERSION}-embeddings.npy")}.')



Running on cpu.
Running on cpu.
Running on cpu.
Running on cpu.
Running on cpu.
Running on cpu.
Running on cpu.
Running on cpu.
Running on cpu.
Running on cpu.
Running on cpu.
100% 3/3 [05:54<00:00, 118.07s/it]
Saved probas, logprobas, logits and embeddings to /content/runs/run_12/output_HCNN_biopython_proteinnet_extra_mols_0p00.csv.
Saved embeddings to /content/runs/run_12/output_HCNN_biopython_proteinnet_extra_mols_0p00-embeddings.npy.
